In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from pyro.contrib.cevae import CEVAE
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

from cate.infra.mlflow import MlflowClient
from cate.model.dataset import Dataset
from cate.model.evaluate import (
    Auuc,
    Outputs,
    QiniByPercentile,
    UpliftByPercentile,
    UpliftCurve,
)
from torch import Tensor
from cate.model.metrics import Artifacts, Metrics
from cate.utils import Timer, get_logger, path_linker

/workspace/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset_name = "test"
logger = get_logger("causalml")
pathlinker = path_linker(dataset_name)
client = MlflowClient("base_pattern")
timer = Timer()

In [ ]:
logger.info("load dataset")
ds = Dataset.load(pathlinker.base)
base_classifier = lgb.LGBMClassifier(
    importance_type="gain", random_state=42, force_col_wise=True, n_jobs=-1
)
base_regressor = lgb.LGBMRegressor(
    importance_type="gain", random_state=42, force_col_wise=True, n_jobs=-1
)

models = {
    "cevae": CEVAE(ds.X.shape[1]).to("cuda"),
}

INFO  2024-11-07 15:29:20 [causalml] load dataset


In [ ]:
skf = StratifiedKFold(5, shuffle=True, random_state=42)
for name, model in models.items():
    logger.info(f"start {name}")
    client.start_run(
        run_name=f"{dataset_name}_{name}",
        tags={"model": name, "dataset": dataset_name, "package": "causalml"},
        description=f"base_pattern: {name} training and evaluation using {dataset_name} dataset with causalml package and lightgbm model with 5-fold cross validation and stratified sampling.",
    )
    client.log_params(
        {
            "importance_type": "gain",
            "random_state": 42,
            "n_jobs": -1,
            "force_col_wise": True,
        }
    )
    _pred_dfs = []
    for i, (train_idx, valid_idx) in tqdm(
        enumerate(skf.split(np.zeros(len(ds)), ds.y))
    ):
        logger.info(f"epoch {i}")
        train_X = ds.X.iloc[train_idx].to_numpy()
        train_y = ds.y.iloc[train_idx].to_numpy().reshape(-1)
        train_w = ds.w.iloc[train_idx].to_numpy().reshape(-1)
        valid_X = ds.X.iloc[valid_idx].to_numpy()
        valid_y = ds.y.iloc[valid_idx].to_numpy().reshape(-1)
        valid_w = ds.w.iloc[valid_idx].to_numpy().reshape(-1)

        timer.start(name, "train", i)
        model.fit(
            Tensor(train_X),
            Tensor(train_w),
            Tensor(train_y),

        )
        timer.stop(name, "train", i)

        timer.start(name, "predict", i)
        pred = model.ite(valid_X)
        timer.stop(name, "predict", i)

        metrics = Metrics(
            list(
                [Auuc()]
                + [UpliftByPercentile(k) for k in np.arange(0, 1, 0.1)]
                + [QiniByPercentile(k) for k in np.arange(0, 1, 0.1)]
            )
        )
        metrics(pred.numpy().reshape(-1), valid_y, valid_w)
        client.log_metrics(metrics, i)

        _pred_dfs.append(
            pd.DataFrame({"index": ds.y.index[valid_idx], "pred": pred.reshape(-1)})
        )

    pred_df = pd.concat(_pred_dfs, axis=0)
    base_df = pd.merge(
        ds.y.rename(columns={ds.y_columns[0]: "y"}),
        ds.w.rename(columns={ds.w_columns[0]: "w"}),
        left_index=True,
        right_index=True,
    )
    output_df = pd.merge(base_df, pred_df, left_index=True, right_index=True)

    artifacts = Artifacts([UpliftCurve(), Outputs()])
    artifacts(output_df.pred.to_numpy(), output_df.y.to_numpy(), output_df.w.to_numpy())
    client.log_artifacts(artifacts)
    client.end_run()

INFO  2024-11-07 15:24:16 [causalml] start cevae


2024/11/07 15:24:16 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
0it [00:00, ?it/s]

INFO  2024-11-07 15:24:17 [causalml] epoch 0


INFO 	 Training with 800 minibatches per epoch
0it [03:59, ?it/s]


KeyboardInterrupt: 